# THIS IS A NEW NOTEBOOK to make poster for AGU 2022

In [ ]:
# Import packages 
import sys
sys.path.append('../Plotting/')
""" Now you can imprt modules in ../Plotting"""
import xyp_plot as xyp

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors


import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import ana as a

import importlib


## Set file names, open data sets, read in

In [ ]:
# Specify directory for data
drct='/glade/p/cesm/amwg_dev/juliob/SAMwrf/Curtains/'
tag='ndg04'
f4=drct+'SAMwrf_'+tag+'_ML_super_v7.nc'
tag='ndg07'
f5=drct+'SAMwrf_'+tag+'_ML_super_v6.nc'
tag='ndg06'
f6=drct+'SAMwrf_'+tag+'_ML_super_v4.nc'
tag='ndg04_nov'
f4n=drct+'SAMwrf_'+tag+'_ML_super_v4.nc'

tag='ndg07'
f7=drct+'SAMwrf_'+tag+'_ML_super_v6.nc'

d4=xr.open_dataset( f4 )
d5=xr.open_dataset( f5 )
d6=xr.open_dataset( f6 )
d4n=xr.open_dataset( f4n )
d7=xr.open_dataset( f7 )

lev=d4['lev']
lon=d4['lon']
lat=d4['lat']

#print(list(d4))

## Access tendencies from datasets and rescale. Access met fields

In [ ]:
# d4 contains results from 'ndg04':
#   - no OGW scheme
#   - "Block" style vertical remapping
utn4=d4['UTEND_NDG']*86400.
vtn4=d4['VTEND_NDG']*86400.
utc4=d4['UTEND_CORE']*86400.
vtc4=d4['VTEND_CORE']*86400.
utgw4=d4['UTEND_GWDTOT']*86400.
vtgw4=d4['VTEND_GWDTOT']*86400.
u4=d4['U']
v4=d4['V']

# d5 contains results from 'ndg05':
#   - Anisotropic OGW scheme
#   - "Block" style vertical remapping
utn5=d5['UTEND_NDG']*86400.
vtn5=d5['VTEND_NDG']*86400.
utc5=d5['UTEND_CORE']*86400.
vtc5=d5['VTEND_CORE']*86400.
utgw5=d5['UTEND_GWDTOT']*86400.
vtgw5=d5['VTEND_GWDTOT']*86400.
u5=d5['U']
v5=d5['V']


# d6 contains results from 'ndg06':
#   - Previous isotropic OGW scheme
#   - "Block" style vertical remapping
utn6=d6['UTEND_NDG']*86400.
vtn6=d6['VTEND_NDG']*86400.
utc6=d6['UTEND_CORE']*86400.
vtc6=d6['VTEND_CORE']*86400.
utgw6=d6['UTEND_GWDTOT']*86400.
vtgw6=d6['VTEND_GWDTOT']*86400.
u6=d6['U']
v6=d6['V']


# d4n contains results from 'ndg04_nov':
#   - No OGW scheme
#   - "None" style vertical remapping
utn4n=d4n['UTEND_NDG']*86400.
vtn4n=d4n['VTEND_NDG']*86400.
utc4n=d4n['UTEND_CORE']*86400.
vtc4n=d4n['VTEND_CORE']*86400.
utgw4n=d4n['UTEND_GWDTOT']*86400.
vtgw4n=d4n['VTEND_GWDTOT']*86400.
u4n=d4n['U']
v4n=d4n['V']

# d7 contains results from 'ndg05':
#   - Anisotropic OGW scheme
#   - "Block" style vertical remapping
#        ***CORRECTed RIDGE ANGLES
utn7=d7['UTEND_NDG']*86400.
vtn7=d7['VTEND_NDG']*86400.
utc7=d7['UTEND_CORE']*86400.
vtc7=d7['VTEND_CORE']*86400.
utgw7=d7['UTEND_GWDTOT']*86400.
vtgw7=d7['VTEND_GWDTOT']*86400.
u7=d7['U']
v7=d7['V']


In [ ]:
ww4=np.sqrt(u4**2+v4**2)
eu4=u4/np.sqrt(u4**2+v4**2)
ev4=v4/np.sqrt(u4**2+v4**2)
chk=eu4**2+ev4**2
print(eu4.shape)


utn_dot_eu = eu4*utn4 + ev4*vtn4 


gu4=np.mean(u4,axis=0)
gu4x,xlon,xlat=a.c_o_xy(idata=gu4,lon=lon,lat=lat,dx=0.25,dy=0.25)

#plt.ylim(-50,50)
plt.ylim(0,1.1)
#plt.plot(ww4[30,:,100],'+')
#plt.plot(u4[30,:,100],'+')
#plt.plot(v4[30,:,100],'+')
plt.plot(chk[30,:,100],'+')
print(ww4[30,:,100].values)
print(u4[30,:,100].values)
print(v4[30,:,100].values)
print(chk[30,:,100].values)

In [ ]:
#calculate 3D pressure for later use
ps=d4['PS']
hyam=d4['hyam']
hybm=d4['hybm']
plev=d4['lev']

gps=np.average( ps, axis=0 )
ghya=np.average( hyam , axis=0 )
ghyb=np.average( hybm , axis=0 )
p3=a.press(PS=gps,hybm=ghyb,hyam=ghya )
p3x,xlon,xlat= a.c_o_xy(idata=p3,lon=lon,lat=lat,dx=0.25,dy=0.25)

In [ ]:

fig = plt.figure(figsize=(20, 9))
cmap = plt.cm.bwr
levs=[29,26,20,10]
ipoo=0
maxws=[20.,30.,40.,80.]
#gutnx=gutnnx
for L in levs:
    maxw=maxws[ipoo]
    ipoo=ipoo+1
    clevs=np.linspace(-maxw,maxw,num=41)

    ax1 = fig.add_subplot(2, 4, ipoo , projection=ccrs.PlateCarree(central_longitude=0))
    cf = ax1.contourf(xlon, xlat, gu4x[L,:,:],  levels=clevs, cmap=cmap  , transform=ccrs.PlateCarree())
    #titl='$\eta$='+str(np.float(plev[L]/1000.))
    titl='$\eta$='+f"{float(plev[L]/1000.):.2f}"

    ax1.coastlines(resolution='110m')
    
    gl=ax1.gridlines(linestyle=':', color='black',draw_labels=True)  #, ylabels_right=None)
    gl.ylabels_right=False
    gl.xlabels_top=False
    
    ax1.set_title( titl ,fontsize=12, loc='center')
    plt.colorbar(cf , ax=ax1 , shrink=0.8,label='$\mathregular{m~s^{-1}}$')

In [ ]:
#means
#gutn=np.mean(utn4,axis=0)
gutn=np.mean(utn_dot_eu,axis=0)
gutnn=np.mean(utn4n,axis=0)
gvtn=np.mean(vtn4,axis=0)
gutgw=np.mean(utgw5,axis=0)
gvtgw=np.mean(vtgw5,axis=0)
gutgwi=np.mean(utgw6,axis=0)
gvtgwi=np.mean(vtgw6,axis=0)
gutnx,xlon,xlat= a.c_o_xy(idata=gutn,lon=lon,lat=lat,dx=0.25,dy=0.25)
gutgwx,xlon,xlat= a.c_o_xy(idata=gutgw,lon=lon,lat=lat,dx=0.25,dy=0.25)

gutnnx,xlon,xlat= a.c_o_xy(idata=gutnn,lon=lon,lat=lat,dx=0.25,dy=0.25)

#std's
vutn=np.std(utn4,axis=0)
vutgw=np.std(utgw5,axis=0)
vutnx,xlon,xlat= a.c_o_xy(idata=vutn,lon=lon,lat=lat,dx=0.25,dy=0.25)
vutgwx,xlon,xlat= a.c_o_xy(idata=vutgw,lon=lon,lat=lat,dx=0.25,dy=0.25)

In [ ]:
clevs=np.linspace(-20.,20.,num=41)
print(clevs)
cmap = plt.cm.bwr
fig = plt.figure(figsize=(20, 9))
levs=[29,26,20,10]
levs=[31,29,20,10]
ipoo=0
#gutnx=gutnnx
for L in levs:
    ipoo=ipoo+1
    
    ax1 = fig.add_subplot(2, 4, ipoo , projection=ccrs.PlateCarree(central_longitude=0))
    cf = ax1.contourf(xlon, xlat, gutnx[L,:,:],  levels=clevs, cmap=cmap  , transform=ccrs.PlateCarree())
    #titl='$\eta$='+str(np.float(plev[L]/1000.))
    titl='$\eta$='+f"{np.float(plev[L]/1000.):.2f}"

    ax1.coastlines(resolution='110m')
    
    gl=ax1.gridlines(linestyle=':', color='black',draw_labels=True)  #, ylabels_right=None)
    gl.ylabels_right=False
    gl.xlabels_top=False
    
    ax1.set_title( titl ,fontsize=12, loc='center')
    plt.colorbar(cf , ax=ax1 , shrink=0.8,label='$\mathregular{m~s^{-1}d^{-1}}$')

for L in levs:
    ipoo=ipoo+1
    
    ax1 = fig.add_subplot(2, 4, ipoo , projection=ccrs.PlateCarree(central_longitude=0))
    cf = ax1.contourf(xlon, xlat, gutnnx[L,:,:],  levels=clevs, cmap=cmap  , transform=ccrs.PlateCarree())
    #titl='$\eta$='+str(np.float(plev[L]/1000.))
    titl='$\eta$='+f"{np.float(plev[L]/1000.):.2f}"

    ax1.coastlines(resolution='110m')
    
    gl=ax1.gridlines(linestyle=':', color='black',draw_labels=True)  #, ylabels_right=None)
    gl.ylabels_right=False
    gl.xlabels_top=False
    
    ax1.set_title( titl ,fontsize=12, loc='center')
    plt.colorbar(cf , ax=ax1 , shrink=0.8,label='$\mathregular{m~s^{-1}d^{-1}}$')


In [ ]:
#importlib.reload(xyp)
clevs=np.linspace(-20.,20.,num=41)
z3x=-7.*np.log(p3x/100_000.)
#xyp.pltxp(gutnx,p3x, xlon[0,:],lats=xlat[:,0],  zlim=[100000.,0.],lat0=-30.)
#plt.contourf(gutnx[:,120,:])
#plt.colorbar()
print(clevs)
xyp.pltxp(gutnx,z3x, xlon[0,:],lats=xlat[:,0],  zlim=[0.,30.],lat0=-30.,xlim=[280.,300.],clevs=clevs)
plt.title("Section at 30S")
plt.xlabel("Longitude")
plt.ylabel("Z(km)")

In [ ]:
#r45,lev,lat,lon = a.corr_utn_utgw(f4,f5)
#r46,lev,lat,lon = a.corr_utn_utgw(f4,f6)
r45 = a.corr_utn_utgw(f4,f5)
r46 = a.corr_utn_utgw(f4,f6)

""" Interpolate to regular 0.25x0.25 grid """
r45x,xlon,xlat= a.c_o_xy(idata=r45,lon=lon,lat=lat,dx=0.25,dy=0.25)
r46x,xlon,xlat= a.c_o_xy(idata=r46,lon=lon,lat=lat,dx=0.25,dy=0.25)

In [ ]:
clevs=np.linspace(-1.,1.,num=41)
print(clevs)
cmap = plt.cm.bwr
fig = plt.figure(figsize=(20, 9))
ipoo=0

levs=[29,26,20,10]
ipoo=0
#gutnx=gutnnx
for L in levs:
    ipoo=ipoo+1
    
    ax1 = fig.add_subplot(2, 4, ipoo , projection=ccrs.PlateCarree(central_longitude=0))
    cf = ax1.contourf(xlon, xlat, r45x[L,:,:],  levels=clevs, cmap=cmap  , transform=ccrs.PlateCarree())
    #titl='$\eta$='+str(np.float(plev[L]/1000.))
    titl='$\eta$='+f"{np.float(plev[L]/1000.):.2f}"

    ax1.coastlines(resolution='110m')
    
    gl=ax1.gridlines(linestyle=':', color='black',draw_labels=True)  #, ylabels_right=None)
    gl.ylabels_right=False
    gl.xlabels_top=False
    
    ax1.set_title( titl ,fontsize=12, loc='center')
    if (ipoo==4):
        plt.colorbar(cf , ax=ax1 , shrink=0.8,label='$correlation$')
    else:
        plt.colorbar(cf , ax=ax1 , shrink=0.8)


In [ ]:
print(np.shape(utgw5))

In [ ]:
fig=plt.figure(figsize=(15,15))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))

ax.add_feature(cartopy.feature.OCEAN, zorder=0)
ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black')

ax.set_global()
ax.gridlines()
plt.xlim(-90,-30)
plt.ylim(-55,10)
L=28
iT=200
#ax.quiver(lon, lat, gutn[L,:],gvtn[L,:] )  #, transform=vector_crs)
#ax.quiver(lon, lat, gutgw[L,:],gvtgw[L,:],scale=250.1 ,color='red')  #, transform=vector_crs)
ax.quiver(lon, lat, utn4[iT,L,:],vtn4[iT,L,:] )  #, transform=vector_crs)
ax.quiver(lon, lat, utgw5[iT,L,:],vtgw5[iT,L,:],scale=250.1 ,color='red')  #, transform=vector_crs)
#ax.quiver(lon, lat, gutgwi[L,:],gvtgwi[L,:] ,scale=50,color='green')  #, transform=vector_crs)



In [ ]:
# d4 contains results from 'ndg04':
#   - no OGW scheme
#   - "Block" style vertical remapping
utn4=d4['UTEND_NDG']*86400.
vtn4=d4['VTEND_NDG']*86400.
utc4=d4['UTEND_CORE']*86400.
vtc4=d4['VTEND_CORE']*86400.
utgw4=d4['UTEND_GWDTOT']*86400.
vtgw4=d4['VTEND_GWDTOT']*86400.
u4=d4['U']
v4=d4['V']

# d5 contains results from 'ndg05':
#   - Anisotropic OGW scheme
#   - "Block" style vertical remapping
utn5=d5['UTEND_NDG']*86400.
vtn5=d5['VTEND_NDG']*86400.
utc5=d5['UTEND_CORE']*86400.
vtc5=d5['VTEND_CORE']*86400.
utgw5=d5['UTEND_GWDTOT']*86400.
vtgw5=d5['VTEND_GWDTOT']*86400.
u5=d5['U']
v5=d5['V']


# d6 contains results from 'ndg06':
#   - Previous isotropic OGW scheme
#   - "Block" style vertical remapping
utn6=d6['UTEND_NDG']*86400.
vtn6=d6['VTEND_NDG']*86400.
utc6=d6['UTEND_CORE']*86400.
vtc6=d6['VTEND_CORE']*86400.
utgw6=d6['UTEND_GWDTOT']*86400.
vtgw6=d6['VTEND_GWDTOT']*86400.
u6=d6['U']
v6=d6['V']


# d4n contains results from 'ndg04_nov':
#   - No OGW scheme
#   - "None" style vertical remapping
utn4n=d4n['UTEND_NDG']*86400.
vtn4n=d4n['VTEND_NDG']*86400.
utc4n=d4n['UTEND_CORE']*86400.
vtc4n=d4n['VTEND_CORE']*86400.
utgw4n=d4n['UTEND_GWDTOT']*86400.
vtgw4n=d4n['VTEND_GWDTOT']*86400.
u4n=d4n['U']
v4n=d4n['V']


In [ ]:

fig2 = plt.figure(figsize=(20, 9))
ipoo=0
for L in levs:
    ipoo=ipoo+1
    
    ax1 = fig2.add_subplot(2, 4, ipoo , projection=ccrs.PlateCarree(central_longitude=0))
    cf = ax1.contourf(xlon, xlat, r46x[L,:],  levels=clevs, cmap=cmap  , transform=ccrs.PlateCarree())

    ax1.coastlines(resolution='110m')
    ax1.gridlines(linestyle=':', color='black')
    ax1.set_title("Plot using pcolormesh",fontsize=12, loc='center')
    plt.colorbar(cf , ax=ax1 , shrink=0.8)


In [ ]:
print(np.shape(d5['hybi']))


utn4=d4['UTEND_NDG']*86400.
gutn4=np.average( np.abs(utn4), axis=0 )
utgw5=d5['UTEND_GWDTOT']*86400.
gutgw5=np.average( np.abs(utgw5), axis=0 ) 

oo=np.where( (gutgw5[28,:]>10.) & (gutn4[28,:]>4))
print(np.shape(oo))

gutgw5x,xlon,xlat= a.c_o_xy(idata=gutgw5,lon=lon,lat=lat,dx=0.25,dy=0.25)


In [ ]:
plt.plot(r45[29,:])
plt.xlim(2440,2450)
fig = plt.figure(figsize=(10, 9))

ax1 = fig.add_subplot(2, 1, 1 ) # , projection=ccrs.PlateCarree(central_longitude=0))
cf = ax1.tricontourf(lon,lat,gutn4[28,:])
ax1.plot(lon[oo[0][:]],lat[oo[0][:]],'x',color='white')
plt.colorbar(cf,ax=ax1)
ax1 = fig.add_subplot(2, 1, 2 ) # , projection=ccrs.PlateCarree(central_longitude=0))
cf = ax1.tricontourf(lon,lat,gutgw5[28,:])
plt.colorbar(cf,ax=ax1)


## 2D histograms of high-freq UTN vs UTGW

In [ ]:
fig = plt.figure(figsize=(18, 9))

y=d5['UTEND_GWDTOT'][0:,27:,:].values *86400. 
x=d4['UTEND_NDG'][0:,27:,:].values *86400.
h2u=np.histogram2d(x.flatten(), y.flatten() ,bins=50 , range=[[-100.,100.],[-100,100]] )


cmap=mpl.colormaps['jet']
#cmap=cm.jet
clevs=[1,3,5,10,30,50,100,300,500,1000,3000,5000 ] #np.logspace(0,3)

ax = fig.add_subplot(1, 2, 1 ) # , projection=ccrs.PlateCarree(central_longitude=0))

ax.set_aspect('equal','box')


cf = ax.contourf(h2u[1][1:],h2u[2][1:],h2u[0],levels=clevs,cmap=cmap,norm=colors.LogNorm())
plt.colorbar(cf,ax=ax,shrink=0.6)
plt.xlabel("$\partial_t U_{OGW}$",fontsize=20)
plt.ylabel("$\partial_t U_{nudging}$",fontsize=20)



In [ ]:
fig = plt.figure(figsize=(18, 9))

delta_core = ( d7['UTEND_CORE'] - d4['UTEND_CORE'] ) *86400.

y=d7['UTEND_GWDTOT'][0:,27:,:].values *86400. 
x=d4['UTEND_NDG'][0:,27:,:].values *86400.  - delta_core[0:,27:,:].values

h2u=np.histogram2d(x.flatten(), y.flatten() ,bins=50 , range=[[-100.,100.],[-100,100]] )


cmap=mpl.colormaps['jet']
#cmap=cm.jet
clevs=[1,3,5,10,30,50,100,300,500,1000,3000,5000 ] #np.logspace(0,3)

ax = fig.add_subplot(1, 2, 1 ) # , projection=ccrs.PlateCarree(central_longitude=0))

ax.set_aspect('equal','box')


cf = ax.contourf(h2u[1][1:],h2u[2][1:],h2u[0],levels=clevs,cmap=cmap,norm=colors.LogNorm())
plt.colorbar(cf,ax=ax,shrink=0.6)
plt.xlabel("$\partial_t U_{OGW}$",fontsize=20)
plt.ylabel("$\partial_t U_{nudging}$",fontsize=20)



In [ ]:

fig2=plt.figure(figsize=(10, 9))
#sca=plt.scatter( gutgw5[28,oo[0][:]],gutn4[28,oo[0][:]])

sca=plt.scatter( d5['UTEND_GWDTOT'][:,30,:]*86400. , d4['UTEND_NDG'][:,30,:]*86400. )
plt.xlim(-100,100)
plt.ylim(-100,100)
utn4.shape
print(r45[29,2446],gps[2446])
#plt.plot(r45[29,:])

In [ ]:
d4['UTEND_GWDTOT'].shape
